### Wireless Insite MIMO Data to CSV
(setup for multiple (MIMO) Tx and multiple (MIMO) Rx)

Last Updated: Spring 2024 - Anant Matta

In [8]:
#Import dependencies
import numpy as np
import scipy as sp
import pandas as pd
import sqlite3

In [9]:
##Sqlite path varaible -- change for appropriate simulation
#Connect to .sqlite data output file
sqlite_file_path = "C:/Users/am122/Desktop/Wireless Insite Assets/Broadband Analyzer Troubleshooting/MIMO_Home_Office_1/Home Office.MIMO_Home_Office_1.sqlite"
conn = sqlite3.connect(sqlite_file_path)

In [10]:
##Power folder path variable (can be exported from wireless insite) -- change for appropriate simulation
# Directory of folder containing 'power' csv files
power_files_path = 'C:/Users/am122/Desktop/Wireless Insite Assets/Broadband Analyzer Troubleshooting/MIMO_Home_Office_1/power'

# Output filename
output_file = 'example_data_formatted.csv'

#Frequency Sweep Parameters (MHz)
start_f = 5710
stop_f = 5730
steps = 21
center_fs = np.linspace(start_f,stop_f,steps)

In [11]:
#View database tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Data tables:")
for table in tables:
    print(table[0])

Data tables:
utd_instance
utd_instance_param
utd_instance_param_type
scene_origin
tx
rx
tx_metadata
rx_metadata
txrx_set_type
rx_set
channel
path
interaction_type
interaction
channel_utd
path_utd
material_property
constant_attenuation_material_property
feature_vertex
diffraction_edge
feature_facet


In [12]:
#To view specific tables
table_name = 'tx'
query = f"SELECT * FROM {table_name}" #Limited to prevent crashing out
temp_df = pd.read_sql_query(query, conn)
temp_df.describe()

,tx_id,tx_set_id,x,y,z
count,1.0,1.0,1.000000,1.00000,1.0
mean,0.0,24.0,-9.446927,-1.98324,1.5
std,NaN,NaN,NaN,NaN,NaN
min,0.0,24.0,-9.446927,-1.98324,1.5
25%,0.0,24.0,-9.446927,-1.98324,1.5
50%,0.0,24.0,-9.446927,-1.98324,1.5
75%,0.0,24.0,-9.446927,-1.98324,1.5
max,0.0,24.0,-9.446927,-1.98324,1.5


In [13]:
table_name = 'tx'
query = f"SELECT * FROM {table_name}" 
tx_locations = pd.read_sql_query(query, conn)

table_name = 'rx'
query = f"SELECT * FROM {table_name}" 
rx_locations = pd.read_sql_query(query, conn)

In [14]:
import csv
import os

#power.txSet{tx_id}.txPt001.rxSet{rx_id}.tx{tx_subelement}.rx{rx_subelement}.inst{instance}
def extract_file_info(filename):
    tokens = filename.split('.')
    tx_id = int(tokens[1].split('txSet')[1])
    rx_id = int(tokens[3].split('rxSet')[1])
    tx_sub = int(tokens[4].split('txEl')[1])
    rx_sub = int(tokens[5].split('rxEl')[1])
    inst = int(tokens[6].split('inst')[1])
    return tx_id,rx_id,tx_sub,rx_sub,inst

data = []

for filename in os.listdir(power_files_path):
    if filename.endswith(".csv"):
        with open(os.path.join(power_files_path, filename), 'r') as file:
            rows = file.readlines()
            tx_id,rx_id,tx_sub,rx_sub,inst = extract_file_info(filename)
            row_num = 0
            for row in rows:
                if not row.startswith("#"): 
                    row = row.strip().split(',')
                    rx_x = rx_locations[rx_locations['rx_set_id']  == rx_id].iloc[row_num]['x']
                    rx_y = rx_locations[rx_locations['rx_set_id']  == rx_id].iloc[row_num]['y']
                    rx_z = rx_locations[rx_locations['rx_set_id']  == rx_id].iloc[row_num]['z']
                    tx_x = tx_locations[tx_locations['tx_set_id']  == tx_id].iloc[0]['x']
                    tx_y = tx_locations[tx_locations['tx_set_id']  == tx_id].iloc[0]['y']
                    tx_z = tx_locations[tx_locations['tx_set_id']  == tx_id].iloc[0]['z']
                    data.append([tx_id,rx_id,tx_sub,rx_sub,center_fs[inst - 1],rx_x,rx_y,rx_z,tx_x,tx_y,tx_z] + row[1:])
                    row_num += 1
            row_num = 0

with open(output_file, 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['tx_id','rx_id','tx_subelement', 'rx_subelement', 'Center Frequency (MHz)','Rx x(m)','Rx y(m)','Rx z(m)','Tx x(m)','Tx y(m)','Tx z(m)'] + ['Received Power (W)', 'Phase (radians)', 'Path Loss (dB)', 'Path Gain (dB)'])
    csv_writer.writerows(data)